# Take streaked flats to characterize dust spots on dispersers
This notebook will take streaked flats of a bright star \
in an attempt to generate flat fields for the dispersers. \
This is the first try, and may require some iteration.
See SITCOM-471 \
Craig Lage - 06Feb23

In [ ]:
import sys
import asyncio
import time
import astropy 
import numpy as np
import logging 
import yaml
import os

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)
# if you want logging
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

### Instantiate the control classes

In [ ]:
domain = salobj.Domain()
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

## Assumes ATCS and LATISS are enabled and the telescope has run prepare_for_on_sky.

# Find a bright target

In addition to the find_target function below, we have the following list of potential targets:\
HD 2261, RA = 0 hrs\
HD 66811 RA = 8 hrs - Best early in the night\
HD 78647 RA = 9 hrs - Best early in the night\
HD 110304 RA = 12 hrs - Best later in the night\
HD 129056 RA = 14 hrs - Best later in the night\
HD 159532A RA = 18\
HD 214952 RA = 22 hrs


In [ ]:
target_name = 'HD 2261'

In [ ]:
target_name = await atcs.find_target(az=90.0,el=70,mag_limit=2.5)
print(target_name)

# Now slew to the object
### This will also set the correct rotator angle.

In [ ]:
await atcs.slew_object(target_name, rot=0.0, rot_type=RotType.SkyAuto)

# Now take an image.
### Verify that the star is centered in the field.  If not, center as needed.

In [ ]:
#await atcs.offset_xy(x=-175, y=-100) # Move to upper left amp
await latiss.take_object(exptime=1.0, n=1, filter='empty_1',grating='empty_1')


# Now stop tracking and take another image.
### This will verify the speed and direction of the trailing.

In [ ]:
await atcs.stop_tracking()
await latiss.take_object(exptime=30.0, n=1, filter='empty_1',grating='empty_1')


# Now slew back to the object, offset it, stop tracking, and take a trailed image.

In [ ]:
await atcs.slew_object(target_name, rot=0.0, rot_type=RotType.SkyAuto)
await atcs.offset_xy(x=-700, y=150) # Move 500 arcsec off the lower right of the field
await atcs.stop_tracking()
await latiss.take_object(exptime=90.0, n=1, filter='empty_1',grating='holo4_003')


# Based on the results, take more images with this object and/or try other objects.